# YOLOv5 Ultralytics

Source:
https://github.com/ultralytics/yolov5

Tutorial:
https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data



# Step 1: Install Requirements

In [2]:
PROJECT_ROOT = "~/code/school/captcha-solver" #@param {type:"string"}

import os.path

# Go to root of project.
%cd {PROJECT_ROOT}

/Users/mve/code/school/captcha-solver


In [2]:
# Only clone repo if not already present
if os.path.isdir("yolov5"):
  print ("File exist")
else:
  !git clone https://github.com/ultralytics/yolov5  # clone repo
  !git checkout apple/mps
%cd yolov5
# %pip install -qr requirements.txt # install dependencies
%pip install --pre -r requirements.txt --extra-index-url https://download.pytorch.org/whl/nightly/cpu # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 13580, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 13580 (delta 75), reused 91 (delta 67), pack-reused 13476bjects:  24% (3260/13580), 1.77 MiB | 807.00 KiB/sReceiving objects:  25% (3397/13580), 2.24 MiB | 818.00 KiB/sReceiving objects:  27% (3667/13580), 2.75 MiB | 839.00 KiB/sReceiving objects:  29% (3939/13580), 2.75 MiB | 839.00 KiB/sReceiving objects:  31% (4210/13580), 2.75 MiB | 839.00 KiB/sReceiving objects:  33% (4559/13580), 3.29 MiB | 857.00 KiB/sReceiving objects:  35% (4753/13580), 3.29 MiB | 857.00 KiB/sReceiving objects:  37% (5025/13580), 3.29 MiB | 857.00 KiB/sReceiving objects:  39% (5297/13580), 3.86 MiB | 882.00 KiB/sReceiving objects:  41% (5568/13580), 3.86 MiB | 882.00 KiB/sReceiving objects:  43% (5840/13580), 3.86 MiB | 882.00 KiB/sReceiving objects:  44% (6039/13580), 4.43 MiB | 913.00 KiB/sReceiving objects:  46% (6247/13580), 4.43 M

In [3]:
%pip install protobuf=="3.20.1" # Fixes an error.

Note: you may need to restart the kernel to use updated packages.


# Step 2: Assemble Our Dataset


In [4]:
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=ultralytics


In [5]:
# set up environment
os.environ["DATASET_DIRECTORY"] = PROJECT_ROOT + "/datasets"

# Change the following code to switch dataset.

In [6]:
# after following the link above, recieve python code with these fields filled in
from roboflow import Roboflow
rf = Roboflow(api_key="lpAPGdHq3w2H0vl0cBAu")
project = rf.workspace("captcha-solver").project("captcha-images")
dataset = project.version(33).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Step 3: Train Our Custom YOLOv5 model

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset locaiton is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training

You can switch weights to any of these pretrained checkpoints:
https://github.com/ultralytics/yolov5#pretrained-checkpoints

# Model info
Tested with 120 epochs, 786 images stretched to 640x640 on a Tesla K80.
- yolov5n6: will take an estimated 1.3 hours
- yolov5s6: will take an estimated 2 hours
- yolov5m6: will take an estimated 5 hours
- yolov5l6: will take an estimated 9 hours
- yolov5x6: Not enough GPU RAM


In [7]:
model = 'yolov5n6.pt' #@param ["yolov5n6.pt", "yolov5s6.pt", "yolov5m6.pt", "yolov5l6.pt", "yolov5x6.pt"] {allow-input: true}

In [8]:
# Install wandb for tracking and visualizing YOLOv5 runs.
!pip install wandb

In [9]:
import wandb
wandb.login()

wandb: Currently logged in as: mve (han-captcha). Use `wandb login --relogin` to force relogin


True

In [10]:
# --resume wandb-artifact://han-captcha/YOLOv5/1182ptzf
# {model}
# /content/best.pt

In [11]:
!python train.py --device mps --img 800 --batch 8 --epochs 5 --data {dataset.location}/data.yaml --weights {model} --cache

wandb: Currently logged in as: mve (han-captcha). Use `wandb login --relogin` to force relogin
train: weights=yolov5n6.pt, cfg=, data=/Users/mve/code/school/captcha-solver/yolov5/~/code/school/captcha-solver/datasets/CAPTCHA-images-33/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=8, imgsz=800, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=mps, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
Traceback (most recent call last):
  File "/Users/mve/code/school/captcha-solver/yolov5/train.py", line 670, in <module>
    main(opt)
  File "/Users/mve/code/sch

# Evaluate Custom YOLOv5 Detector Performance
Training losses and performance metrics are saved to Tensorboard and also to a logfile.

If you are new to these metrics, the one you want to focus on is `mAP_0.5` - learn more about mean average precision [here](https://blog.roboflow.com/mean-average-precision/).

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [12]:
# Use newly run weights.
!python detect.py --weights runs/train/exp/weights/best.pt --img 800 --conf 0.1 --source {dataset.location}/test/images

detect: weights=['runs/train/exp/weights/best.pt'], source=/Users/mve/code/school/captcha-solver/yolov5/~/code/school/captcha-solver/datasets/CAPTCHA-images-33/test/images, data=data/coco128.yaml, imgsz=[800, 800], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-202-g7d46c69 Python-3.9.1 torch-1.11.0 CPU

Traceback (most recent call last):
  File "/Users/mve/code/school/captcha-solver/yolov5/detect.py", line 252, in <module>
    main(opt)
  File "/Users/mve/code/school/captcha-solver/yolov5/detect.py", line 247, in main
    run(**vars(opt))
  File "/Users/mve/miniforge3/envs/yolov5-ultralytics/lib/python3.9/site-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    retu

In [13]:
# Use existing weights.
# !python detect.py --weights /content/v11-yolov5s6.pt --img 800 --conf 0.1 --source {dataset.location}/test/images

In [5]:
# display inference on ALL test images

import glob
from IPython.display import Image, display

# for imageName in glob.glob(PROJECT_ROOT + '/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
for imageName in glob.glob(PROJECT_ROOT + '/images/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# Conclusion and Next Steps

Congratulations! You've trained a custom YOLOv5 model to recognize your custom objects.

To improve you model's performance, we recommend first interating on your datasets coverage and quality. See this guide for [model performance improvement](https://github.com/ultralytics/yolov5/wiki/Tips-for-Best-Training-Results).

To deploy your model to an application, see this guide on [exporting your model to deployment destinations](https://github.com/ultralytics/yolov5/issues/251).

Once your model is in production, you will want to continually iterate and improve on your dataset and model via [active learning](https://blog.roboflow.com/what-is-active-learning/).

In [15]:
# Export your model's weights for future use
# from google.colab import files
# files.download('./runs/train/exp/weights/best.pt')